In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
from ipywidgets import fixed, Layout, Button, Box
import kmc.variables
import inspect
import kmc.morphology
import kmc.rates

def get_classes(modulo,atributo):
    morphs = []
    for name, value in vars(modulo).items():
        if '__' not in name and hasattr(value, atributo):
            morphs.append((name,value))
    return morphs

for name, value in vars(kmc.morphology).items():
    if '__' not in name and hasattr(value, 'make'):
        print(name,value, type(value), inspect.getfullargspec(value)[0])


Lattice <class 'kmc.morphology.Lattice'> <class 'type'> ['self', 'num_sites', 'vector', 'disorder', 'composition']
Lattice_BHJ <class 'kmc.morphology.Lattice_BHJ'> <class 'type'> ['self', 'n_loops', 'cutoff', 'num_sites', 'vector', 'disorder', 'composition']
Bilayer <class 'kmc.morphology.Bilayer'> <class 'type'> ['self', 'axis', 'num_sites', 'vector', 'disorder', 'composition']
Saw <class 'kmc.morphology.Saw'> <class 'type'> ['self', 'size_saw', 'width_saw', 'num_sites', 'vector', 'disorder', 'composition']


In [2]:
def basic_widget(**kwargs):
    global basic
    basic = '###BASIC PARAMETERS###########################################################\n'
    for keys in kwargs.keys():
        basic += f'{keys} = {kwargs[keys]}\n'
    basic +='##############################################################################'    
    #print(ss)
    #return ss    

def morph_write(**kwargs):
    global text
    value = kwargs['morphology']
    text = '###MORPHOLOGY#################################################################\n'
    text += f"morphology ={value.__name__}("
    args = [kwargs[i] for i in kwargs.keys() if i != 'morphology']
    text += ','.join(args)
    text += ')\n'
    text += '#############################################################################'
    global composition
    composition = kwargs['composition']
    #print(text)
    #return text

def morph_widget(**kwargs):
    WIDS, kw = [], {}
    value = kwargs['morphology']
    kw['morphology'] = fixed(value)
    print(inspect.getdoc(value))
    for arg in inspect.getfullargspec(value)[0]:
        if arg != 'self':
            wid = widgets.Text(
            value='',
            placeholder='Type something',
            description=arg.title(),
            disabled=False
            ) 
            WIDS.append(wid)
            kw[arg] = wid      
    box1 = widgets.HBox(WIDS)
    display(box1)
    write_wid = widgets.interactive_output(morph_write,kw)
    display(write_wid)
    

BOOLS, STRS = [], []
kw = {}

for name, value in vars(kmc.variables).items():
    if '__' not in name:
        if isinstance(value, bool):
            wid = widgets.Checkbox(
            value=value,
            description=name.title(),
            disabled=False
            )
            BOOLS.append(wid)
            kw[name] = wid
        elif isinstance(value, str):
            wid = widgets.Text(
            value=value,
            placeholder='Type something',
            description=name.title(),
            disabled=False
            )
            STRS.append(wid)
            kw[name] = wid
        elif isinstance(value, float):
            wid = widgets.FloatText(
            value=value,
            description=name.title(),
            disabled=False
            )    
            STRS.append(wid)
            kw[name] = wid
        elif isinstance(value, int):
            wid = widgets.IntText(
            value=value,
            description=name.title(),
            disabled=False
            )    
            STRS.append(wid)
            kw[name] = wid    


morphs = get_classes(kmc.morphology,'make')

Morph = widgets.Dropdown(
options=morphs,
value=morphs[0][1],
# rows=10,
description='Morphology:',
disabled=False
)

box1   = widgets.HBox(BOOLS) 
box2   = widgets.HBox(STRS) 
display(box1,box2)
wid_basic = widgets.interactive_output(basic_widget,kw)
display(wid_basic) 
display(Morph)
wid_morph = widgets.interactive_output(morph_widget,{'morphology':Morph})
display(wid_morph)


def energy_write(**kwargs):
    global text
    text += '\n#ENERGIES\n#Gaussian distribuitions\n'
    for key in kwargs.keys():
        text += f'morphology.Gaussian_energy({kwargs[key]})\n'    


def energies():
    WIDS,b2 = [], []
    kw = {}
    num_mats = len(composition.split(','))
    for level in ['s1', 't1', 'homo', 'lumo']:
        t    = [str(i)+':(mean,std)' for i in range(num_mats)]
        t    = ','.join(t)
        t    = '{'+t+'}'
        wid = widgets.Text(
        value=t,
        description=level,
        disabled=False
        )            
        WIDS.append(wid)
        kw[level] = wid
        box1   = widgets.HBox(WIDS)
        b2.append(box1)
        WIDS = []
    box1   = widgets.VBox(b2)    
    display(box1)        
    write_wid = widgets.interactive_output(energy_write,kw)
    display(write_wid)




Output()

Dropdown(description='Morphology:', options=(('Lattice', <class 'kmc.morphology.Lattice'>), ('Lattice_BHJ', <c…

Output()

Output()

In [7]:
energies()
print(basic)
print(text)

Output()

###BASIC PARAMETERS###########################################################
identifier = teste
animation_mode = False
save_animation = False
animation_exten = gif
time_limit = 10000
pause = False
marker_type = 1
rotate = False
frozen_lattice = False
bimolec = False
periodic = False
n_proc = 1
rounds = 1
##############################################################################
###MORPHOLOGY#################################################################
morphology =Lattice(1000,[5,5,0],[0,0,0],[0.5,0.5])
#############################################################################
#ENERGIES
#Gaussian distribuitions
morphology.Gaussian_energy({0:(mean,std),1:(mean,std)})
morphology.Gaussian_energy({0:(mean,std),1:(mean,std)})
morphology.Gaussian_energy({0:(mean,std),1:(mean,std)})
morphology.Gaussian_energy({0:(mean,std),1:(mean,std)})



In [4]:

rates = get_classes(kmc.rates,'action')
Rates = widgets.Dropdown(
options=rates,
value=rates[0][1],
# rows=10,
description='Processes:',
disabled=False
)

display(Rates)

Dropdown(description='Processes:', options=(('Forster', <class 'kmc.rates.Forster'>), ('ForsterT', <class 'kmc…

In [5]:
rates = get_classes(kmc.morphology,'assign_to_system')
rates = [i for i in rates if i not in morphs]
Rates = widgets.Dropdown(
options=rates,
value=rates[0][1],
# rows=10,
description='Morphology:',
disabled=False
)

display(Rates)

Dropdown(description='Morphology:', options=(('Create_Particles', <class 'kmc.morphology.Create_Particles'>), …

In [6]:
a = Rates.value
inspect.getfullargspec(a)

FullArgSpec(args=['self', 'kind', 'num', 'method'], varargs=None, varkw='kwargs', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})